In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import numpy as np
import csv
import os
import pandas as pd

In [ ]:
width = 256 
height = 192 
channels = 3

fin_dir = '../datasets/FinUI/images'
fin_img = os.listdir(fin_dir)
fin_img = [fin_dir + '/' + fi for fi in fin_img if fi.split('.')[1]=='png']

X = []
  
for image in fin_img:
    X.append(cv2.resize(cv2.imread(image, cv2.IMREAD_COLOR), (width, height), interpolation=cv2.INTER_AREA))
    
X_fin = np.array(X)

ex_s = pd.read_csv('../datasets/FinUI/100_avg_scores.csv')
y_fin = ex_s.mean(axis=1).values
# y_fin = ex_s['3_1'].values


X_train = X_fin[:70, :]
y_train = y_fin[:70]

X_val = X_fin[70:, :]
y_val = y_fin[70:]

ntrain = len(X_train)
nval = len(X_val)

In [ ]:
from keras import layers
from keras import models
from keras import optimizers
from keras import regularizers
from keras import initializers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
import h5py

def traverse_datasets(hdf_file):

    def h5py_dataset_iterator(g, prefix=''):
        for key in g.keys():
            item = g[key]
            path = f'{prefix}/{key}'
            if isinstance(item, h5py.Dataset): # test for dataset
                yield (path, item)
            elif isinstance(item, h5py.Group): # test for group (go down)
                yield from h5py_dataset_iterator(item, path)

    with h5py.File(hdf_file, 'r') as f:
        for path, _ in h5py_dataset_iterator(f):
            yield path
            
weights = {}
filename = 'flickr_style.h5'

with h5py.File(filename, 'r') as f:
    for dset in traverse_datasets(filename):
        weights[dset] = f[dset][:]

conv1_bias = weights['/conv1/conv1/bias:0']
conv1_kernel = weights['/conv1/conv1/kernel:0']
conv2_bias = weights['/conv2/conv2/bias:0']
conv2_kernel = weights['/conv2/conv2/kernel:0']
conv3_bias = weights['/conv3/conv3/bias:0']
conv3_kernel = weights['/conv3/conv3/kernel:0']
conv4_bias = weights['/conv4/conv4/bias:0']
conv4_kernel = weights['/conv4/conv4/kernel:0']
conv5_bias = weights['/conv5/conv5/bias:0']
conv5_kernel = weights['/conv5/conv5/kernel:0']


In [ ]:
from keras.layers import Layer
from keras import backend as K

class LRN(Layer):
    
    def __init__(self, n=5, alpha=0.0001, beta=0.75, k=2, **kwargs):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LRN, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LRN, self).build(input_shape)

    def call(self, x, mask=None):
        if K.image_data_format == "th":
            _, f, r, c = self.shape
        else:
            _, r, c, f = self.shape
        half_n = self.n // 2
        squared = K.square(x)
        pooled = K.pool2d(squared, (half_n, half_n), strides=(1, 1),
                         padding="same", pool_mode="avg")
        if K.image_data_format == "th":
            summed = K.sum(pooled, axis=1, keepdims=True)
            averaged = (self.alpha / self.n) * K.repeat_elements(summed, f, axis=1)
        else:
            summed = K.sum(pooled, axis=3, keepdims=True)
            averaged = (self.alpha / self.n) * K.repeat_elements(summed, f, axis=3)
        denom = K.pow(self.k + averaged, self.beta)
        return x / denom
    
    def get_output_shape_for(self, input_shape):
        return input_shape
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "n": self.n,
            "alpha": self.alpha,
            "beta": self.beta,
            "k": self.k
        })
        return config

In [ ]:
l = 0.001 # weight decay

input_shape = (192, 256, 3)

im_data = layers.Input(shape=input_shape, dtype='float32', name='im_data')

conv1 = layers.Conv2D(96, kernel_size=(11, 11), strides=(4, 4), name='conv1', 
                        activation='relu', input_shape=input_shape, 
                        kernel_regularizer=regularizers.l2(l))(im_data)

pool1 = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)
norm1 = LRN(name="norm1")(pool1)
drop1 = layers.Dropout(0.1)(norm1)

layer1_1 = layers.Lambda(lambda x: x[:, :, :, :48])(drop1)
layer1_2 = layers.Lambda(lambda x: x[:, :, :, 48:])(drop1)

conv2_1 = layers.Conv2D(128, kernel_size=(5, 5), strides=(1, 1),
                        activation='relu',
                        padding='same', 
                        name='conv2_1', 
                        kernel_regularizer=regularizers.l2(l))(layer1_1)

conv2_2 = layers.Conv2D(128, kernel_size=(5, 5), strides=(1, 1),
                        activation='relu',
                        padding='same', 
                        name='conv2_2',
                        kernel_regularizer=regularizers.l2(l))(layer1_2)

conv2 = layers.Concatenate(name='conv_2')([conv2_1, conv2_2])

pool2 = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv2)
norm2 = LRN(name="norm2")(pool2)
drop2 = layers.Dropout(0.1)(norm2)

conv3 = layers.Conv2D(384, kernel_size=(3, 3), strides=(1, 1), activation='relu', 
                        name='conv3',
                        padding='same',
                        kernel_regularizer=regularizers.l2(l))(drop2)
drop3 = layers.Dropout(0.1)(conv3)

layer3_1 = layers.Lambda(lambda x: x[:, :, :, :192])(drop3)
layer3_2 = layers.Lambda(lambda x: x[:, :, :, 192:])(drop3)

conv4_1 = layers.Conv2D(192, kernel_size=(3, 3), strides=(1, 1),
                        activation='relu', 
                        padding='same',
                        name='conv4_1',
                        kernel_regularizer=regularizers.l2(l))(layer3_1)

conv4_2 = layers.Conv2D(192, kernel_size=(3, 3), strides=(1, 1),
                        activation='relu', 
                        padding='same',
                        name='conv4_2',
                        kernel_regularizer=regularizers.l2(l))(layer3_2)

conv4 = layers.Concatenate(name='conv_4')([conv4_1, conv4_2])

layer4_1 = layers.Lambda(lambda x: x[:, :, :, :192])(conv4)
layer4_2 = layers.Lambda(lambda x: x[:, :, :, 192:])(conv4)

conv5_1 = layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1),
                        activation='relu',
                        padding='same', 
                        name='conv5_1',
                        kernel_regularizer=regularizers.l2(l))(layer4_1)

conv5_2 = layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1),
                        activation='relu',
                        padding='same', 
                        name='conv5_2',
                        kernel_regularizer=regularizers.l2(l))(layer4_2)

conv5 = layers.Concatenate(name='conv_5')([conv5_1, conv5_2])

pool5 = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(conv5)

flat = layers.Flatten()(pool5)
fc6 = layers.Dense(1024, activation='relu', name='fc6',
                        kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.01),
                        bias_initializer='zeros',
                        kernel_regularizer=regularizers.l2(l))(flat)
drop6 = layers.Dropout(0.5)(fc6)

fc7 = layers.Dense(512, activation='relu', name='fc7', 
                        kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.01),
                        bias_initializer='zeros',
                        kernel_regularizer=regularizers.l2(l))(drop6)
drop7 = layers.Dropout(0.5)(fc7)

fc8 = layers.Dense(1, name='fc8',
                        kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.01),
                        bias_initializer='zeros')(drop7)

model = models.Model(inputs=im_data, outputs=fc8)

In [ ]:
model.get_layer('conv1').set_weights([conv1_kernel[:, :, :, :], conv1_bias[:]])
model.get_layer('conv2_1').set_weights([conv2_kernel[:, :, :, :128], conv2_bias[:128]])
model.get_layer('conv2_2').set_weights([conv2_kernel[:, :, :, 128:], conv2_bias[128:]])
model.get_layer('conv3').set_weights([conv3_kernel[:, :, :, :], conv3_bias[:]])
model.get_layer('conv4_1').set_weights([conv4_kernel[:, :, :, :192], conv4_bias[:192]])
model.get_layer('conv4_2').set_weights([conv4_kernel[:, :, :, 192:], conv4_bias[192:]])
model.get_layer('conv5_1').set_weights([conv5_kernel[:, :, :, :128], conv5_bias[:128]])
model.get_layer('conv5_2').set_weights([conv5_kernel[:, :, :, 128:], conv5_bias[128:]])


# model.summary()

In [ ]:
import visualkeras
from PIL import ImageFont

In [ ]:
visualkeras.layered_view(model,legend=True, draw_volume=True, font=ImageFont.truetype("arial.ttf", 16))

In [ ]:
def rmse(y_true, y_pred):
	return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def euclidean_distance_loss(y_true, y_pred):
    return 0.5 * K.mean(K.square(y_pred - y_true), axis=-1)

In [ ]:
batch_size = 20

train_datagen = ImageDataGenerator(rescale = 1./255)                                  
val_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

In [ ]:
epochs = 150
decay = 0.001 
base_lr = 0.01

sgd = optimizers.SGD(learning_rate=base_lr, momentum=0.9, decay=decay, nesterov=True)
model.compile(loss=euclidean_distance_loss , optimizer=sgd, metrics=[rmse])

In [ ]:
history = model.fit(
    train_generator,                          
    steps_per_epoch = ntrain // batch_size,                           
    epochs = epochs,                          
    validation_data=val_generator,                         
    validation_steps=nval // batch_size,
    verbose=0
)

In [ ]:
rmse = history.history["rmse"]
val_rmse = history.history["val_rmse"]

epochs_x = range(1, len(rmse) + 1)

In [ ]:
plt.plot(epochs_x, rmse, 'b', label='Training RMSE')
plt.plot(epochs_x, val_rmse, 'r', label='Validation RMSE')

plt.legend()
plt.show()

In [ ]:
from scipy import stats

def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy
    Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''
    N = len(x)
    r, p = stats.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(N-3)
    z = stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    return r, p, lo, hi

In [ ]:
predictions = []

X_test = X_fin / 255.0
y_test = y_fin
for img in X_test:
  img = img.reshape(1, 192, 256, 3)
  pred = model.predict(img, verbose=0)
  predictions.append(float(pred))

predictions = np.array(predictions)

In [ ]:
from numpy.polynomial.polynomial import polyfit
b, m = polyfit(y_test, predictions, 1)

fig = plt.figure()
plt.scatter(y_test, predictions, c='c')
plt.plot(y_test, b + m * y_test, '-', c='b')
plt.xlabel('User ratings', labelpad=10, fontsize=16)
plt.ylabel('Predicted ratings', labelpad=10, fontsize=16)

plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

corr, p, lo, hi = pearsonr_ci(y_test, predictions)
print('Pearsons correlation: r=%.2f, p=%.2e, CI=[%.2f, %.2f]' % (corr, p, lo, hi))
rmse_test = sqrt(mean_squared_error(y_test, predictions))
print('RMSE: %.3f' % rmse_test)

In [ ]:
corr, p, lo, hi = pearsonr_ci(y_val, predictions)
print('Pearsons correlation: r=%.2f, p=%.2e, CI=[%.2f, %.2f]' % (corr, p, lo, hi))
rmse_test = sqrt(mean_squared_error(y_val, predictions))
print('RMSE: %.3f' % rmse_test)